**Splunk Windows Security Log Analysis**

1. Project Overview and Goals
Purpose: analyze Windows Security Logs using Splunk to develop a behavioral baseline and investigate host activity patterns.

Emphasis on Layer 1 of your six-layer host OS anatomy model: Process Execution.

2. Ingest Phase – Validate Log Availability in Splunk
2A. Identify Active Index: Determine where logs are stored and volume available.

2B. Verify Key Event Codes: Confirm presence of 4688, 4624, 4672, etc.

2C. Validate Field Extractions: Ensure proper parsing of fields (e.g., Account_Name, CommandLine).

Design Note: Decision to retain high-fidelity, high-volume logging for educational purposes.

3. Baselining – Understand Normal Behavior
3A. Logon Patterns (EventCode=4624): Break down logon types and accounts.

3B. Process Creation Relationships (EventCode=4688): Parent-child process pairs to identify common behavior.

3C. Process Type Breakdown: Analyze interactive vs. system vs. other types.

Interpretation: System idle for 10 days, resumed with standard startup and trusted process behavior only.

4. Process Execution – Pattern and Anomaly Review
4A. Most Frequently Launched Processes: Volume-based ranking of New_Process_Name.

4B. Parent-Child Chains: Deeper inspection of process lineages.

4C. Rare Process Executions (≤ 3 times): Anomaly check – confirmed no interactive or low-frequency launches.

Conclusion: Strong baseline, no irregular executions, aligns with controlled lab conditions.



**RESULTS:**

 2. Ingest Phase: Validate Log Availability in Splunk

 A. Goal of This Phase

Confirm that Windows Security log data is available in Splunk, that key event types relevant to user logon and process behavior are being ingested, and that critical fields are being parsed correctly. This ensures we can continue building behavior-based detection logic in later phases.

---

### B. Actions Taken

1. **Identified the Active Index:**
   - Ran the following query to find where the Windows Security logs were stored:
     ```spl
     index=* sourcetype="WinEventLog:Security"
     | stats count by index
     ```
   - Result showed that ~65,000 Security events were present in the **`main`** index.

2. **Verified the Presence of Key Event Codes:**
   - Ran the following query to analyze which types of Windows Security events were present:
     ```spl
     index=main sourcetype="WinEventLog:Security"
     | stats count by EventCode
     | sort -count
     ```
   - Findings:
     - Majority of volume was from system-generated events (e.g., `4703`, `4689`, `5447`)
     - Crucially, important behavioral events were present:
       - `4688` – Process creation ✅
       - `4624` – Successful logon ✅
       - `4672` – Special privileges assigned ✅
       - `4625`, `4648` – Credential-related ✅

3. **Validated Field Extractions:**
   - Ran sample queries to inspect whether Splunk correctly extracted fields like `Account_Name`, `New_Process_Name`, and `CommandLine`:
     ```spl
     index=main sourcetype="WinEventLog:Security"
     | table _time, host, EventCode, Account_Name, Logon_Type, New_Process_Name, CommandLine
     | head 20
     ```

   - Confirmed that:
     - Fields like `_time`, `host`, `EventCode`, and `Account_Name` were properly parsed.
     - Process creation events (4688) included values for `New_Process_Name` and occasionally `CommandLine`.
     - Most activity observed was generated by **Splunk’s internal processes** (`splunk-optimize.exe`), not interactive users.

### C. Interpretation and Outcome

The large event volume (~65,000 events) does **not** represent log backfill from the time the machine was powered off. A targeted search confirmed that no logs were generated or present in the event store between June 18 and June 25, which aligns with the known system downtime.

Instead, the observed events were all generated on June 27, during the short window after the machine was powered on. This volume reflects typical startup behavior, background system processes, and Splunk internal activity — including frequent process creation, termination, service events, and system auditing.

The system’s audit policies appear to be configured for high-fidelity logging (e.g., process tracking and firewall changes), resulting in rapid log accumulation even during idle or service-driven operation.


 D. Next Steps

With ingest validation complete, we will now proceed to:

**Section 3: Baselining Normal Behavior**

This phase will establish typical logon and process creation patterns on the system, so that later phases can focus on deviations from this baseline.

 Design Note: Data Volume and Verbosity Decision


 Design Note: Data Volume and Verbosity Decision

As part of validating log ingestion in Section 2, we observed that more than **65,000 Security Event Log entries** were generated within just a few hours of system uptime. These included high-frequency event types such as:

- `4688` – Process creation
- `4689` – Process exit
- `4703` – Privilege adjustments
- `5447` – Windows Filtering Platform changes

This prompted a critical question:  
> **Is this level of logging — with tens of thousands of events in a few hours — desirable or efficient for the project, or should it be reduced?**

---
**Decision Framework**

We have decided to **retain this high-volume configuration** for the current project **for learning and visibility purposes**. The reasons are as follows:

1. **Educational Value.**  
   Seeing this level of verbosity firsthand helps develop analyst intuition about:
   - How fast logs can accumulate under audit-heavy conditions.
   - What types of noise are common in host-level telemetry.
   - The difference between service/system-generated events vs. interactive or malicious behavior.

2. **Detection Engineering Foundation.**  
   In upcoming phases (baselining, detection, and triage), we will need to:
   - Separate signal from noise.
   - Define what "normal" looks like in a noisy environment.
   - Understand how and why false positives occur.

3. **Controlled Environment.**  
   This is a lab environment, not a production system.  
   We are not constrained by ingestion licensing or storage limits, and we accept the overhead as part of the learning process.

---

**What the 65,000 Events Represent**

These events were not backfilled from a time the system was powered off. We confirmed through targeted time-bounded searches that **no logs were generated during that powered-off period**.

Instead, the 65,000+ events represent a **burst of activity upon system startup**, including background services, Splunk internal processes, Windows filtering, and privilege tracking. These types of events are typical in systems with detailed audit policies enabled.

---

**Future Plan**

In a real-world deployment or a production-tuned simulation, this configuration would be adjusted to:

- Filter out low-value event types (e.g., `4689`)
- Focus on high-fidelity signals (e.g., `4688` with suspicious command lines, `4624` interactive logons)
- Reduce ingestion cost, noise, and analyst fatigue

After this project is completed, we will conduct a follow-up version with a **streamlined audit configuration** and compare event volume, detection surface, and analytic clarity.

**Section 3: Baselining – Understanding Normal Behavior**
Purpose
This phase establishes a behavioral baseline of the system by examining typical login and process creation patterns. A solid baseline enables the analyst to more easily distinguish legitimate activity from potential threats during the triage and detection phases that follow.

✅ 3A: Logon Patterns by User and Host
Objective
Identify who is logging in and how, based on EventCode=4624 (successful logons), broken down by account and logon type.


index=main EventCode=4624
| stats count by Account_Name, Logon_Type, host
Results (Sample)
Account_Name	Logon_Type	Host	Count
Steve	2	DESKTOP-UPKRV33	4
SYSTEM	5	DESKTOP-UPKRV33	128
DESKTOP-UPKRV33$	2, 5	DESKTOP-UPKRV33	132

Interpretation
Steve (Type 2): Indicates interactive local logins. Though user believes they only logged in once, system activity and session handling may generate multiple entries (e.g., fast user switching, automatic reauthentication).

SYSTEM and machine accounts (Type 5): Expected for background services and scheduled tasks.

No unexpected users were observed.

✅ 3B: Process Creation Patterns (Parent-Child Relationships)
Objective
Baseline what processes are launching on the system by examining parent and child process relationships from EventCode=4688.


index=main EventCode=4688
| rex "Creator Process Name:\s+(?P<Parent_Process_Name>[^\r\n]+)"
| rex "New Process Name:\s+(?P<New_Process_Name>[^\r\n]+)"
| stats count by Parent_Process_Name, New_Process_Name, host
| sort -count
Highlights from Results
~22,800+ process creation events.

Top child processes included:

cmd.exe, conhost.exe, python.exe, reg.exe, powershell.exe (likely user-initiated or script-driven)

svchost.exe, SearchIndexer.exe, dllhost.exe (background services)

chrome_installer_uncompressed.exe, mpam-d.exe, update platform binaries (software auto-updates)

Interpretation
The system spawns a wide range of expected processes from trusted parents (services.exe, svchost.exe, cmd.exe, etc.). No rogue process paths or anomalous patterns were identified during this initial survey.

3C. Interpretation and Outcome
The process activity telemetry for the last 7 days reveals 24,656 events, with the following high-level breakdown:

"Other" process type: 22,849 events

"Interactive" process type: 1,409 events

"System" process type: 398 events

Despite the relatively high number of interactive-classified processes, this does not reflect active analyst-driven scripting or manual command-line use. The system was powered off for approximately 10 days and was only turned on within the current analysis window. User activity has been limited exclusively to a single login and the initiation of this Splunk-based project.

Therefore:

The "Interactive" activity likely stems from system startup behavior, application autostarts, and trusted software initialization routines (e.g., background launch of cmd.exe, powershell.exe, or python.exe associated with system services or third-party software like Slack, OneDrive, or Python environments).

The "Other" and "System" categories are consistent with normal background operations such as Windows service launches, scheduled tasks, telemetry collectors, and trusted process chains like svchost.exe spawning other system binaries.

This baseline reinforces that a high volume of telemetry does not necessarily imply malicious or suspicious activity. Instead, it highlights how automated system and service behavior can dominate endpoint logs, especially upon system boot.

**Section 4 – Understand Normal Behavior: Process Creation Patterns** 
In this section, we examined host process creation telemetry to understand standard execution behavior across system components and user activity. This is a critical part of baselining host activity under Layer 1: Process Execution of our six-layer host operating system anatomy model.

4A. Determine Most Frequently Launched Processes
Using EventCode=4688 (new process creation events), we aggregated all launched processes by New_Process_Name.

Findings:

The most frequently executed processes are:

splunk-optimize.exe – over 22,000 executions, driven by Splunk’s internal indexing and optimization jobs.

Other high-frequency processes include:

splunk-powershell.exe

splunkd.exe

splunk-MonitorNoHandle.exe

splunk-netmon.exe

svchost.exe and various Windows service-related binaries

These are primarily associated with:

Splunk system operations

Core Windows services

Background processes tied to Defender, indexing, and telemetry collection

This confirms that the majority of process activity is system-generated, not interactive.

4B. Analyze Parent-Child Process Relationships
We examined process lineage by extracting both Parent_Process_Name and New_Process_Name fields from the Message field in 4688 events.

Findings:

The dominant parent-child relationship was:

splunkd.exe spawning splunk-optimize.exe – repeated extensively across internal Splunk maintenance operations.

Other common chains include:

Splunk spawning Python and PowerShell subprocesses

Microsoft Edge and VS Code launching subordinate processes (e.g., python.exe, cmd.exe, reg.exe)

System services like services.exe and svchost.exe spawning background Windows processes

This confirms that the host is behaving as a telemetry-forwarding system, with no signs of abnormal execution chains.

4C. Identify Rare Process Executions (≤ 3 times)
We attempted to isolate rare or potentially anomalous process launches by filtering for EventCode=4688 entries where a process was created 3 times or fewer in the 7-day window.

Result:

No qualifying results were returned.

This means that all recorded processes executed more than three times in the observed period.

Possible interpretations:

Low-interaction system with only automated recurring processes.

The absence of interactive, one-off commands aligns with your reported system use: the host was offline for 10 days, and only reactivated for this project.

Conclusion for Section 4
This phase confirms that the system exhibits stable, automated background behavior, with a clear process baseline dominated by Splunk indexing and Windows services. There is no evidence of irregular or low-frequency process activity, supporting the assumption that the host is currently in a known and controlled state.











✅ **Conclusion: Findings and Forward Strategy**

This project served as an initial deep-dive into host-based telemetry using Splunk and Windows Security logs. The focus was on validating log ingestion, establishing behavioral baselines, and understanding how system processes and user activity are reflected in Security EventCodes — particularly 4624 (logon) and 4688 (process creation).

**🔍 What We Accomplished**
We completed three critical phases of host telemetry analysis:

**Section 2: Log Ingest Validation**
Confirmed that logs are successfully flowing into Splunk from the host system.

Verified that key EventCodes (4624, 4688, 4672, 4625) were present and parsed correctly.

Observed that over 65,000 log events were generated in a single day — despite minimal user activity — due to detailed audit policies and Splunk’s own internal operations.

**Section 3: Baselining Logon and Process Behavior**
Identified normal system users (Steve, SYSTEM, and the computer account).

Noted that most 4624 logons were system-driven and that user Steve logged in only once, yet multiple entries were generated due to background handling.

Cataloged 4688-based process creation patterns showing Splunk as the dominant generator (splunk-optimize.exe, splunkd.exe, etc.).

Classified process telemetry into Interactive, System, and Other categories, confirming that startup services and background processes made up the vast majority of execution activity.

**Section 4: Process Creation Frequency and Lineage**
Aggregated the most frequent process executions.

Mapped parent-child process relationships to verify trusted lineage (e.g., svchost.exe spawning expected services).

Searched for low-frequency, potentially anomalous processes — but none were found due to the host’s limited activity.

⚠️ **What We Discovered**
The project revealed a fundamental inefficiency in the current configuration:

Excessive log volume is being generated automatically, dominated by:

System services (e.g., svchost.exe)

Audit-intensive events (e.g., process starts/stops)

Splunk's own internal operations

Low signal-to-noise ratio for manual or analyst-driven activity

Difficulty separating background noise from meaningful behavior without significant filtering and interpretation

🛠️ **Why Configuration Changes Are Now Required**
The project proved that while verbose logging can be educational, this level of data volume is not sustainable for efficient, repeatable analysis. Therefore, in future projects, we will:

Streamline Windows audit policies to reduce verbosity (e.g., suppress event codes like 4689 for process termination).

Focus logging on high-value signals, including:

Interactive logons (Logon_Type=2 or 10)

Script execution (e.g., suspicious PowerShell, cmd.exe, or python.exe)

Scheduled task registration or registry changes

Reduce internal telemetry from Splunk where possible, for cleaner baselining.

🧭 **Final Value of This Project**

Although no abnormal behavior was found, the real value of this effort lies in what it taught:

Detection engineering begins not with finding threats, but with learning how to collect the right data.

By grappling with raw, noisy telemetry and establishing what “normal” looks like, we have set a solid foundation for:

Building high-fidelity detection logic

Designing cleaner triage playbooks

Optimizing future audit configurations to better balance signal clarity with data volume

Next steps will include documenting revised audit configurations and re-running streamlined versions of this analysis to compare efficiency, signal density, and investigative clarity.